In [2]:
import pypyodbc as odbc
import pandas as pd

DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'DESKTOP-I0V76P2\SQLEXPRESS' 
DATABASE_NAME = 'foodie_fi'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection=yes;
    # uid=;
    # pwd=;
"""

def execute_query_to_df(query):
    """Executes a SQL query and returns the results as a pandas DataFrame."""

    conn = odbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=cursor.description)
    df.columns = [col[0] for col in df.columns]
    cursor.close()
    conn.close()
    return df

In [2]:
query = """
SELECT * FROM plans
"""

execute_query_to_df(query)

,plan_id,plan_name,price
0,0,trial,0.00
1,1,basic monthly,9.90
2,2,pro monthly,19.90
3,3,pro annual,199.00
4,4,churn,None


In [3]:
query = """
SELECT TOP (5) * FROM subscriptions
"""

execute_query_to_df(query)

,customer_id,plan_id,start_date
0,1,0,2020-08-01
1,1,1,2020-08-08
2,2,0,2020-09-20
3,2,3,2020-09-27
4,3,0,2020-01-13


B. Data Analysis Questions

1. How many customers has Foodie-Fi ever had?

2. What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value

3. What plan start_date values occur after the year 2020 for our dataset? Show the breakdown by count of events for each plan_name

4. What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

5. How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?

6. What is the number and percentage of customer plans after their initial free trial?

7. What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?
8. How many customers have upgraded to an annual plan in 2020?

9. How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?

10. Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)

11. How many customers downgraded from a pro monthly to a basic monthly plan in 2020?

In [4]:
# 1. How many customers has Foodie-Fi ever had?

query = """
    SELECT
        COUNT(DISTINCT customer_id)
    FROM subscriptions
"""

execute_query_to_df(query)

,
0,1000


In [5]:
# 2. What is the monthly distribution of trial plan start_date values for our dataset - 
# use the start of the month as the group by value

query = """
    SELECT
        MONTH(s.start_date) AS start_month,
        COUNT(s.customer_id) AS count_of_trials
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
    WHERE p.plan_name = 'trial'
    GROUP BY MONTH(s.start_date)
    ORDER BY start_month ASC;
"""

execute_query_to_df(query)

,start_month,count_of_trials
0,1,88
1,2,68
2,3,94
3,4,81
4,5,88
5,6,79
6,7,89
7,8,88
8,9,87
9,10,79


In [6]:
# 3. What plan start_date values occur after the year 2020 for our dataset? Show the breakdown
# by count of events for each plan_name

query = """
    SELECT
        p.plan_name, 
        COUNT(s.plan_id) AS count_of_plans
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
    WHERE YEAR(s.start_date) > '2020'
    GROUP BY p.plan_name
"""

execute_query_to_df(query)

,plan_name,count_of_plans
0,basic monthly,8
1,churn,71
2,pro annual,63
3,pro monthly,60


In [8]:
# 4. What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

query = """
    SELECT
        COUNT(DISTINCT s.customer_id) AS count_of_churn,
        CAST(
            CAST(COUNT(DISTINCT s.customer_id) AS FLOAT) / 
            CAST((SELECT COUNT(DISTINCT customer_id) FROM subscriptions) AS FLOAT) 
                * 100 
            AS DECIMAL(10,1)) AS pct_of_churn
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON s.plan_id = p.plan_id
    WHERE p.plan_name = 'churn'
"""

execute_query_to_df(query)

,count_of_churn,pct_of_churn
0,307,30.7


In [9]:
# 5. How many customers have churned straight after their initial free trial - what percentage
# is this rounded to the nearest whole number?

query = """
    WITH cte AS(
        SELECT 
            s.customer_id, 
            p.plan_name,
            s.start_date,
            ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.start_date) AS rownum
        FROM subscriptions AS s
            INNER JOIN plans AS p
                ON p.plan_id = s.plan_id
                )
            
    SELECT 
        COUNT(customer_id) AS ct_churn_after_trial,
        CAST(
            CAST(COUNT(customer_id) AS FLOAT) /
            CAST((SELECT COUNT(DISTINCT customer_id) FROM subscriptions) AS FLOAT) 
                * 100
            AS DECIMAL(10,1)) AS pct_churn_after_trial
    FROM cte WHERE plan_name = 'churn' AND rownum = 2
"""

execute_query_to_df(query)

,ct_churn_after_trial,pct_churn_after_trial
0,92,9.2


In [10]:
# 6. What is the number and percentage of customer plans after their initial free trial?

query = """
    WITH cte AS(
        SELECT 
            s.customer_id, 
            p.plan_name,
            s.start_date
        FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
            )
            
    SELECT 
        COUNT(DISTINCT customer_id) AS ct_plans_after_trial,
        CAST(
            CAST(COUNT(DISTINCT customer_id) AS FLOAT) /
            CAST((SELECT COUNT(DISTINCT customer_id) FROM subscriptions) AS FLOAT) 
                * 100
            AS DECIMAL(10,1)) AS pct_plans_after_trial
    FROM cte 
    WHERE plan_name <> 'churn' AND plan_name <> 'trial'
"""

execute_query_to_df(query)

,ct_plans_after_trial,pct_plans_after_trial
0,908,90.8


In [11]:
# 7. What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?

query = """
WITH cte AS(
        SELECT 
            s.customer_id,
            s.start_date,
            p.plan_name,
            ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.start_date DESC) AS rownum
        FROM subscriptions AS s 
            INNER JOIN plans AS p
                ON p.plan_id = s.plan_id
        WHERE s.start_date <= '20201231'
        )
        
    SELECT
        plan_name,
        COUNT(customer_id) AS plan_count,
        CAST(
            CAST(COUNT(customer_id) AS FLOAT) / 
            CAST(SUM(COUNT(customer_id)) OVER() AS FLOAT) 
                * 100
            AS DECIMAL(10,1)) AS plan_pct
    FROM cte
    WHERE rownum = 1
    GROUP BY plan_name
"""

execute_query_to_df(query)

,plan_name,plan_count,plan_pct
0,churn,236,23.6
1,pro annual,195,19.5
2,basic monthly,224,22.4
3,pro monthly,326,32.6
4,trial,19,1.9


In [12]:
# 8. How many customers have upgraded to an annual plan in 2020?

query = """
    SELECT
        COUNT (DISTINCT s.customer_id)
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON s.plan_id = p.plan_id
    WHERE s.start_date >= '20200101' AND s.start_date <= '20201231'
    AND p.plan_name = 'pro annual'
"""

execute_query_to_df(query)

## The above is just a count of users that went into an annual plan in 2020, could this be that they
## are automatically put into an annual plan following a trial period? If someone upgraded, then
## it would be from one (already paid for) plan to another.

,
0,195


In [13]:
# 8. How many customers have upgraded to an annual plan in 2020?

query = """
WITH cte AS(
    SELECT
        s.customer_id,
        p.plan_name,
        s.start_date,
        ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY start_date ASC) AS rownum
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON s.plan_id = p.plan_id
    WHERE s.start_date >= '20200101' AND s.start_date <= '20201231'
    AND p.plan_name <> 'trial' AND p.plan_name <> 'churn'
)

SELECT 
    COUNT(DISTINCT customer_id) 
FROM cte 
WHERE rownum > 1 AND plan_name = 'pro annual'
"""

execute_query_to_df(query)

# The above firstly filters out any that are trial or churn, this is so that we are only concerned with
# paid-for plans. Then finds the numbers of upgrades from any plan to the pro annual plan.

,
0,158


In [14]:
# 9. How many days on average does it take for a customer to upgrade to  an annual plan from 
# the day they join Foodie-Fi?

query = """
WITH cte AS(
    SELECT
        s.customer_id,
        p.plan_name,
        s.start_date,
        DATEDIFF(day,
                FIRST_VALUE(start_date) 
                        OVER(PARTITION BY customer_id 
                        ORDER BY start_date ASC), 
                s.start_date) AS upgrade_to_annual
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
            )
            
SELECT AVG(upgrade_to_annual) FROM cte WHERE plan_name = 'pro annual'
"""

execute_query_to_df(query)

,
0,104


In [15]:
# Different method for the above question

query = """
SELECT AVG(upgrade_to_annual) FROM (SELECT
                                        DATEDIFF(day,
                                                (SELECT MIN(s1.start_date) FROM subscriptions AS s1
                                                WHERE s1.customer_id = s.customer_id), 
                                                s.start_date) AS upgrade_to_annual
                                    FROM subscriptions AS s
                                        INNER JOIN plans AS p
                                            ON p.plan_id = s.plan_id
                                    WHERE plan_name = 'pro annual') AS a
"""

execute_query_to_df(query)

,
0,104


In [16]:
# 10. Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)

query = """
WITH cte AS(
    SELECT
        DATEDIFF(day,
                (SELECT MIN(s1.start_date) FROM subscriptions AS s1
                WHERE s1.customer_id = s.customer_id), 
                s.start_date) AS upgrade_to_annual
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
    WHERE plan_name = 'pro annual'
    )

SELECT 
    CASE
        WHEN upgrade_to_annual <= 30 THEN '0-30 days'
        WHEN upgrade_to_annual <= 60 THEN '031-60 days'
        WHEN upgrade_to_annual <= 90 THEN '061-90 days'
        WHEN upgrade_to_annual <= 120 THEN '091-120 days'
        WHEN upgrade_to_annual <= 150 THEN '121-150 days'
        WHEN upgrade_to_annual <= 180 THEN '151-180 days'
        WHEN upgrade_to_annual <= 210 THEN '181-210 days'
        WHEN upgrade_to_annual > 210 THEN '210+ days'
    END AS days_to_upgrade,
    COUNT(cte.upgrade_to_annual) AS count_of_upgrades
FROM cte
GROUP BY 
    CASE
        WHEN upgrade_to_annual <= 30 THEN '0-30 days'
        WHEN upgrade_to_annual <= 60 THEN '031-60 days'
        WHEN upgrade_to_annual <= 90 THEN '061-90 days'
        WHEN upgrade_to_annual <= 120 THEN '091-120 days'
        WHEN upgrade_to_annual <= 150 THEN '121-150 days'
        WHEN upgrade_to_annual <= 180 THEN '151-180 days'
        WHEN upgrade_to_annual <= 210 THEN '181-210 days'
        WHEN upgrade_to_annual > 210 THEN '210+ days'
    END
"""

execute_query_to_df(query)

,days_to_upgrade,count_of_upgrades
0,0-30 days,49
1,031-60 days,24
2,061-90 days,34
3,091-120 days,35
4,121-150 days,42
5,151-180 days,36
6,181-210 days,26
7,210+ days,12


In [11]:
# 11. How many customers downgraded from a pro monthly to a basic monthly plan in 2020?

query = """
WITH cte AS(
    SELECT
        s.customer_id,
        p.plan_name,
        s.start_date,
        ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.start_date) AS row_num
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
    WHERE (p.plan_name = 'basic monthly' OR p.plan_name = 'pro monthly')
    AND s.start_date >= '20200101' AND s.start_date <= '20201231'
    )
    
SELECT COUNT(DISTINCT customer_id) FROM cte WHERE row_num = 2 AND plan_name = 'pro monthly'
"""

execute_query_to_df(query)

,
0,163


The Foodie-Fi team wants you to create a new payments table for the year 2020 that includes amounts paid by each customer in the subscriptions table with the following requirements:

* monthly payments always occur on the same day of month as the original start_date of any monthly paid plan
* upgrades from basic to monthly or pro plans are reduced by the current paid amount in that month and start immediately
* upgrades from pro monthly to pro annual are paid at the end of the current billing period and also starts at the end of the month period
* once a customer churns they will no longer make payments

In [254]:
query = """
DECLARE @startdate date = '20200101';
DECLARE @enddate date = '20201231';

-- Creating the full list of potential payments. Recursive union
-- with DATEADD against each customer_id from the original start date
-- of each subscription to the max date of the period (@enddate)
WITH recursive_subs AS(
    SELECT 
        s.plan_id,
        s.customer_id, 
        s.start_date AS payment_date, 
        p.plan_name, 
        p.price 
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id    
    WHERE p.plan_name <> 'trial'
    AND p.plan_name <> 'churn'
    
    UNION ALL
    
    SELECT 
        rs.plan_id,
        rs.customer_id, 
        CASE WHEN rs.plan_name = 'pro annual'
            THEN DATEADD(year, 1, rs.payment_date)
            ELSE DATEADD(month, 1, rs.payment_date) END,
        rs.plan_name, 
        rs.price 
    FROM recursive_subs AS rs
        INNER JOIN plans AS p
            ON p.plan_id = rs.plan_id
    WHERE DATEADD(month, 1, rs.payment_date) <= @enddate
    AND p.plan_name <> 'trial'
    AND p.plan_name <> 'churn'
    ),
 
-- Table that produces a date in the column "limit_date" that
-- confirms the max date of the current subscription type,
-- i.e. the date at which the customer switched to another
-- subscription tier (generally an upgrade or churn)
plan_limit AS(
    SELECT 
        s.customer_id,
        p.plan_name,
        LEAD(s.start_date) 
            OVER(
                PARTITION BY s.customer_id 
                ORDER BY s.start_date) AS limit_date 
    FROM subscriptions AS s
        INNER JOIN plans AS p
            ON p.plan_id = s.plan_id
    WHERE plan_name <> 'trial'
    ),

-- As subscription payments are always paid on the same day of the
-- month, to coincide with the first payment date, this table confirms
-- that day.
upgrade_day AS(
    SELECT
    *,
    DATEPART(day, 
                LEAD(max_date) 
                    OVER(
                        PARTITION BY customer_id 
                        ORDER BY max_date)) AS upgrade_day
    FROM (
        SELECT 
            rs.customer_id, 
            rs.plan_name, 
            MAX(rs.payment_date) AS max_date
        FROM recursive_subs AS rs
        GROUP BY rs.customer_id, rs.plan_name
        ) AS i
),

-- This table introduces some of the logic required to present
-- the correct values:
-- payment_date:this corrects the payment date of any upgrade to 
--              reflect the correct day of the month to pay on
-- lead_plan:   this column provides a way of filtering the data in the next query
--              i.e. if the next row is 'pro annual' then we wish to filter this current 
--              row from the data set
-- lag_price:   this column provides a way of calculating the correct payment cost,
--              i.e. annual plans are reduced by the cost of the previous payment
main AS(
    SELECT 
        rs.plan_id,
        rs.customer_id,
        DATEADD(day,
                COALESCE(ud.upgrade_day - DATEPART(day, rs.payment_date), 0),
                rs.payment_date) AS payment_date,
        rs.plan_name,
        rs.price,
        LEAD(rs.plan_name) 
            OVER(
                PARTITION BY rs.customer_id 
                ORDER BY rs.plan_id, rs.payment_date) AS lead_plan,
        LAG(rs.price)
            OVER(
                PARTITION BY rs.customer_id
                ORDER BY rs.plan_id, rs.payment_date) AS lag_price
    FROM recursive_subs AS rs
        LEFT OUTER JOIN plan_limit AS pl
            ON rs.customer_id = pl.customer_id
            AND rs.plan_name = pl.plan_name
        LEFT OUTER JOIN upgrade_day AS ud
            ON rs.customer_id = ud.customer_id
            AND rs.plan_name = ud.plan_name
    WHERE rs.payment_date <= @enddate
    AND (rs.payment_date <= pl.limit_date OR pl.limit_date IS NULL)
)

-- Using the 'helper' columns created a bove to filter the data further,
-- and calculate the correct price for each payment date.
SELECT 
    customer_id,
    plan_id,
    plan_name,
    payment_date,
    CASE 
        WHEN lag_price IS NULL 
            THEN price
            ELSE
                (CASE 
                WHEN plan_name = 'pro annual' 
                    THEN price-lag_price 
                    ELSE price END)
        END AS amount,
    ROW_NUMBER() 
        OVER(PARTITION BY customer_id 
        ORDER BY payment_date) AS payment_order
FROM main
WHERE lead_plan IS NULL OR lead_plan <> 'pro annual'
ORDER BY customer_id, payment_date
"""

execute_query_to_df(query)

,customer_id,plan_id,plan_name,payment_date,amount,payment_order
0,1,1,basic monthly,2020-08-08,9.90,1
1,1,1,basic monthly,2020-09-08,9.90,2
2,1,1,basic monthly,2020-10-08,9.90,3
3,1,1,basic monthly,2020-11-08,9.90,4
4,1,1,basic monthly,2020-12-08,9.90,5
...,...,...,...,...,...,...
4363,999,2,pro monthly,2020-10-30,19.90,1
4364,999,2,pro monthly,2020-11-30,19.90,2
4365,1000,2,pro monthly,2020-03-26,19.90,1
4366,1000,2,pro monthly,2020-04-26,19.90,2


### D. Outside The Box Questions
The following are open ended questions which might be asked during a technical interview for this case study - there are no right or wrong answers, but answers that make sense from both a technical and a business perspective make an amazing impression!

1. How would you calculate the rate of growth for Foodie-Fi?
2. What key metrics would you recommend Foodie-Fi management to track over time to assess performance of their overall business?
3. What are some key customer journeys or experiences that you would analyse further to improve customer retention?
4. If the Foodie-Fi team were to create an exit survey shown to customers who wish to cancel their subscription, what questions would you include in the survey?
5. What business levers could the Foodie-Fi team use to reduce the customer churn rate? How would you validate the effectiveness of your ideas?

In [288]:
# 1. How would you calculate the rate of growth for Foodie-Fi?

# MOM Growth, YOY Growth, Actual and Percentage.

query = """
    DECLARE @maxdate DATE = (SELECT MAX(start_date) FROM subscriptions);
    
    WITH cte AS(
        SELECT 
            s.customer_id,
            s.start_date,
            CASE 
                WHEN p.plan_name = 'pro annual' 
                    THEN DATEADD(month, 12, start_date) 
                    ELSE (
                        CASE WHEN LEAD(s.start_date) OVER(
                                        PARTITION BY s.customer_id
                                        ORDER BY s.start_date) IS NULL
                            THEN GETDATE()
                            ELSE LEAD(s.start_date) OVER(
                                        PARTITION BY s.customer_id
                                        ORDER BY s.start_date)
                        END
                    )
                END AS end_date,
            p.plan_name,
            CAST(
                CASE 
                    WHEN p.plan_name = 'pro annual' 
                        THEN p.price / 12 
                        ELSE p.price 
                    END AS decimal(10,2)) AS monthly_price
        FROM subscriptions AS s
            INNER JOIN plans AS p
                ON p.plan_id = s.plan_id
    ),
    
    cte2 AS(
        SELECT 
            *,
            DATEDIFF(month, start_date, end_date) AS plan_age,
            CAST(
                monthly_price * DATEDIFF(month, start_date, end_date) AS decimal(10,2)) AS plan_revenue
        FROM cte
        ),
        
    cte3 AS(    
    SELECT 
        DATEADD(month, increment, (SELECT MIN(start_date) FROM cte2)) AS month,
        LEAD(DATEADD(month, increment, (SELECT MIN(start_date) FROM cte2))) OVER(ORDER BY DATEADD(month, increment, (SELECT MIN(start_date) FROM cte2))) AS lagged_month
    FROM 
        (SELECT number AS increment
        FROM master..spt_values 
        WHERE type='P' 
        AND number<DATEDIFF(month, (SELECT MIN(cte2.start_date) FROM cte2), 
                                    (SELECT MAX(cte2.end_date) FROM cte2))) AS DatesList
        )
    
    SELECT 
        FORMAT(month, 'yyyy-MM') AS month,
        SUM(plan_revenue) AS revenue,
        SUM(plan_revenue) - LAG(SUM(plan_revenue)) 
                                    OVER(ORDER BY month) AS mom_growth_act,
        CAST(
            ((SUM(plan_revenue) / 
            LAG(SUM(plan_revenue)) 
                    OVER(ORDER BY month)) *100)
            -100 AS DECIMAL(10,2)) AS mom_growth_pct,
        SUM(plan_revenue) - LAG(SUM(plan_revenue), 12) 
                                    OVER(ORDER BY month) AS yoy_growth_act,
        CAST(
            ((SUM(plan_revenue) / 
            LAG(SUM(plan_revenue), 12) 
                    OVER(ORDER BY month)) *100)
            -100 AS DECIMAL(10,2)) AS yoy_growth_pct
    FROM cte3
        INNER JOIN cte2
            ON start_date <= month
            AND end_date > lagged_month
    WHERE month <= @maxdate
    GROUP BY month, lagged_month
    ORDER BY month;
"""

execute_query_to_df(query)

,month,revenue,mom_growth_act,mom_growth_pct,yoy_growth_act,yoy_growth_pct
0,2020-02,21402.02,None,None,None,None
1,2020-03,43589.22,22187.20,103.67,None,None
2,2020-04,71499.14,27909.92,64.03,None,None
3,2020-05,94458.42,22959.28,32.11,None,None
4,2020-06,120057.90,25599.48,27.10,None,None
5,2020-07,141707.94,21650.04,18.03,None,None
6,2020-08,168673.70,26965.76,19.03,None,None
7,2020-09,204496.74,35823.04,21.24,None,None
8,2020-10,235957.84,31461.10,15.38,None,None
9,2020-11,263215.70,27257.86,11.55,None,None


In [305]:
# 2. What key metrics would you recommend Foodie-Fi management to track over time 
# to assess performance of their overall business?

# Conversion rate (trial to paid conversion rate)
query = """
WITH rownumbered AS(
    SELECT 
        s.customer_id,
        p.plan_name,
        ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.start_date) AS rownum
    FROM subscriptions AS s 
        INNER JOIN plans as P 
            ON s.plan_id = p.plan_id
            )
            
SELECT 
    CAST(
        COUNT(customer_id) / 
        CAST((SELECT COUNT(customer_id) FROM rownumbered WHERE rownum = 1) AS DECIMAL(10,2)) 
        * 100 AS DECIMAL(10,2))
FROM rownumbered 
WHERE rownum = 2 AND plan_name <> 'churn'
"""

execute_query_to_df(query)

,
0,90.80


In [362]:
# Retention Rate
query = """
    DECLARE @maxdate DATE = (SELECT MAX(start_date) FROM subscriptions);
    DECLARE @retainedmonths INT = 3;
    
WITH rownumbered AS(
    SELECT 
        s.customer_id,
        p.plan_name,
        s.start_date,
        
        (SELECT MIN(s1.start_date) 
        FROM subscriptions AS s1 
            INNER JOIN plans AS p1 
                ON s1.plan_id = p1.plan_id 
        WHERE p1.plan_name <> 'trial' 
        AND s1.customer_id = s.customer_id) AS min_start_date,
        
        (SELECT MAX(s1.start_date) 
        FROM subscriptions AS s1 
            INNER JOIN plans AS p1 
                ON s1.plan_id = p1.plan_id 
        WHERE p1.plan_name <> 'trial' 
        AND p1.plan_name = 'churn' 
        AND s1.customer_id = s.customer_id) AS churn_date
        
    FROM subscriptions AS s 
        INNER JOIN plans as P 
            ON s.plan_id = p.plan_id
    WHERE p.plan_name <> 'trial'
            ),

retention_calc AS(                
    SELECT 
        customer_id, 
        MAX(start_date) AS start_date, 
        MAX(churn_date) AS churn_date,
        MAX(CASE 
            WHEN churn_date IS NULL
                THEN DATEDIFF(month, min_start_date, DATEADD(month, 2, @maxdate))
                ELSE DATEDIFF(month, min_start_date, churn_date)
            END) AS retention  
    FROM rownumbered
    GROUP BY customer_id
    )

SELECT
    COUNT(customer_id) AS total_cucstomers,
    
    (SELECT COUNT(customer_id) 
    FROM retention_calc 
    WHERE retention >= @retainedmonths) AS customers_retained,
    
    CAST((SELECT COUNT(customer_id) 
        FROM retention_calc 
        WHERE retention >= @retainedmonths) / 
    CAST(COUNT(customer_id) AS DECIMAL(10,2)) 
        * 100 AS DECIMAL(10,2)) AS retention_pct
FROM retention_calc
"""

execute_query_to_df(query)

,total_cucstomers,customers_retained,retention_pct
0,1000,832,83.20
